In [1]:
import random
import pandas as pd
import numpy as np
import os
import random
import string
import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option('display.max_columns', 64)
pd.set_option('display.max_rows', 100)

In [2]:
# 读入并合并数据
homePath = 'game'
trainPath = os.path.join(homePath, 'train.csv')
# testPath = os.path.join(homePath, 'test.csv')
trainData = pd.read_csv(trainPath)
# testData = pd.read_csv(testPath)

# trainLabel = trainData['acc_now_delinq']
# trainData.drop('acc_now_delinq', axis=1, inplace=True)
# allData = pd.concat([trainData, testData], axis=0, ignore_index=True)

E:\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
subTrain = trainData.loc[trainData['acc_now_delinq'] == 1]
trainData_copy = trainData.copy()
delLocal = np.array(np.where(np.array(trainData['acc_now_delinq']) == 1))
for i in delLocal.tolist():
    trainData_copy.drop(i, axis=0, inplace=True)
trainSize = len(subTrain)
randIndex = random.sample(range(trainSize, len(trainData_copy)), 5*trainSize)
subTrain = pd.concat([subTrain, trainData_copy.loc[randIndex]], axis=0, ignore_index=True)

notBlankIndex = np.where(np.array(subTrain['member_id']) > 0)
subTrain = subTrain.loc[notBlankIndex][:13172]
allData = subTrain.reset_index()

E:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in greater
  # Remove the CWD from sys.path while we load stuff.


In [4]:
# term 贷款的支付次数。数值以几个月为单位，可以是36或60
termData = allData['term']
termData = termData.str.split(' ', n=2, expand=True)[1]
termData = list(map(int, termData))
allData['term'] = termData

In [5]:
# grade LC分配的贷款等级
gradeData = allData['grade']
gradeData.replace(['A','B','C','D','E','F','G'], [7,6,5,4,3,2,1],inplace=True)
allData['grade'] = gradeData*100

In [6]:
# sub_grade LC指定的贷款第二等级
subGradeData = allData['sub_grade']
subGradeData.replace(['A1','A2','A3','A4','A5','B1','B2','B3','B4','B5','C1','C2','C3','C4','C5','D1','D2','D3','D4','D5','E1','E2','E3','E4','E5','F1','F2','F3','F4','F5','G1','G2','G3','G4','G5'],
                    [35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1], inplace=True)
allData['sub_grade'] = subGradeData*30

In [7]:
# emp_title 借款人申请贷款时提供的职位。*
empTitleData = np.array(allData['emp_title'])
empTitleData2 = list(allData['emp_title'])
empTitleData3 = allData['emp_title']
empSet = set(empTitleData2)
times = 0
for i in empSet:
    local = np.where(empTitleData == i)
    count = empTitleData2.count(i)
    empTitleData3.loc[local] = count
    times += 1
    if times % 1000 == 0:
        print(times)
print('ok')
# 保存，以防意外停止造成丢失
emp = pd.DataFrame(empTitleData)
emp.to_csv('empTitle.csv', index=False)

E:\Anaconda3\envs\tensorflow\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


1000
2000
3000
4000
5000
6000
7000
ok


In [8]:
# # ********读取
# empTitle = pd.read_csv('empTitle.csv')
# allData['emp_title'] = empTitle

In [9]:
# emp_length 就业年限。可能的值在0到10之间，其中0表示少于一年，10表示十年或更多年
empLengthData = allData['emp_length']
empLengthData.replace(['< 1 year', '1 year', '2 years', '3 years', '4 years', '5 years', '6 years', '7 years', '8 years', '9 years', '10+ years', 'n/a'],
                     [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0], inplace=True)
allData['emp_length'] = empLengthData*10

In [10]:
# home_ownership 借款人在注册期间提供的房屋所有权状况。
allData['home_ownership'].replace(['MORTGAGE', 'RENT', 'OWN', 'OTHER', 'NONE', 'ANY'], [443557, 356117, 87470, 182, 50, 3], inplace=True)

In [11]:
# verification_status 指示共同借款人的联合收入是否通过LC验证，未验证，或收入来源是否经过验证
allData['verification_status'].replace(['Source Verified', 'Verified', 'Not Verified'], [329558, 291071, 266750], inplace=True)


In [12]:
# issue_d 贷款的月份
issueDData = allData['issue_d']
issueDValueDict = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
issueDValue = list()
for issueD in issueDData:
    issueSplit = issueD.split('-')
    month = issueDValueDict[issueSplit[0]]
    year = int(issueSplit[1])
    issueDValue.append(year*100 + month)
allData['issue_d'] = issueDValue

In [13]:
# loan_status 贷款的当前状态
allData['loan_status'].replace(['Current', 'Fully Paid', 'Charged Off', 'Late (31-120 days)', 'Issued', 'In Grace Period',
                                'Late (16-30 days)', 'Does not meet the credit policy. Status:Fully Paid',
                               'Default', 'Does not meet the credit policy. Status:Charged Off'], [601779, 207723, 45248, 11591, 8460, 6253,
                                                                                                   2357, 1988, 1219, 761], inplace=True)


In [14]:
# pymnt_plan 指示是否已经为贷款实施了付款计划
allData['pymnt_plan'].replace(['n', 'y'], [0, 1], inplace=True)


In [15]:
# desc 贷款说明由借款人提供
allData.drop('desc', axis=1, inplace=True)

In [16]:
# purpose 借款人为贷款申请提供的类别。
allData['purpose'].replace(['debt_consolidation', 'credit_card', 'home_improvement', 'other', 'major_purchase', 'small_business', 'car',
                            'medical', 'moving', 'vacation', 'house', 'wedding', 'renewable_energy', 'educational'], [524215, 206182, 51829,
                                42894, 17277, 10377, 8863, 8540, 5414, 4736, 3707, 2347, 575, 423], inplace=True)

In [17]:
# title 借款人提供的贷款头衔
titleData = np.array(allData['title'])
titleData2 = list(allData['title'])
titleData3 = allData['title']
titleSet = set(titleData2)
for i in titleSet:
    local = np.where(titleData == i)
    count = titleData2.count(i)
    titleData3.loc[local] = count
title = pd.DataFrame(titleData)
title.to_csv('title.csv', index=False)

E:\Anaconda3\envs\tensorflow\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [18]:
# # ************读取
# title = pd.read_csv('title.csv')
# allData['title'] = title

In [19]:
# zip_code 借款人在贷款申请中提供的前3个邮政编码
zipCodeData = np.array(allData['zip_code'])
zipCodeData2 = list(allData['zip_code'])
zipCodeData3 = allData['zip_code']
zipCodeSet = set(zipCodeData2)
for i in zipCodeSet:
    local = np.where(zipCodeData == i)
    count = zipCodeData2.count(i)
    zipCodeData3.loc[local] = count
zipCodeData3.to_csv('zipcode.csv', index=False)

E:\Anaconda3\envs\tensorflow\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [20]:
# # *******读取
# zipCode = pd.read_csv('zipcode.csv')
# allData['zip_code'] = zipCode

In [21]:
# addr_state 借款人在贷款申请中提供的州
addrStateData = np.array(allData['addr_state'])
addrStateData2 = list(allData['addr_state'])
addrStateSet= set(addrStateData2)
addrStateData3 = allData['addr_state']
for i in addrStateSet:
    local = np.where(addrStateData == i)
    count = addrStateData2.count(i)
    addrStateData3.loc[local] = count
allData['addr_state'] = addrStateData3

E:\Anaconda3\envs\tensorflow\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [22]:
# earliest_cr_line 借款人最早报告的信用额度被打开的月份
earliestCrLineData = allData['earliest_cr_line']
earliestCrLineData = earliestCrLineData.fillna('Sep-99999990')
earliestCrLineDict = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
earliestCrLineValue = list()
for ecl in earliestCrLineData:
    eclSplit = ecl.split('-')
    eclMonth = earliestCrLineDict[eclSplit[0]]
    eclYear = int(eclSplit[1])
    earliestCrLineValue.append(eclYear*100 + eclMonth)

allData['earliest_cr_line'] = pd.DataFrame(earliestCrLineValue * 10)
print('ok')


ok


In [23]:
# initial_list_status 贷款的初始上市状态。可能的值是 - W，F
allData['initial_list_status'].replace(['f','w'], [1, 0], inplace=True)

In [24]:
# application_type 表明贷款是个人申请还是与两名共同借款人共同申请
allData['application_type'].replace(['INDIVIDUAL', 'JOINT'], [886868, 511], inplace=True)

In [25]:
# verification_status_joint 指示共同借款人的共同收入是否经LC核实，未核实，或收入来源是否经核实。
allData['verification_status_joint'].replace(['Not Verified', 'Verified', 'Source Verified'], [283, 167, 61], inplace=True)

In [26]:
trainLabel = allData['acc_now_delinq']

In [27]:
allData['emp_title'].fillna(allData['emp_title'].median(), inplace=True)

In [28]:
allData['tot_coll_amt'].fillna(allData['tot_coll_amt'].median(), inplace=True)

In [29]:
allData['total_rev_hi_lim'].fillna(allData['total_rev_hi_lim'].median(), inplace=True)

In [30]:
allData['tot_cur_bal'].fillna(allData['tot_cur_bal'].median(), inplace=True)

In [31]:
allData.fillna(99999999, inplace=True)

In [32]:
trainData = allData[:6586]
testData = allData[6586:]
testData.to_csv(os.path.join(homePath, 'test_n.csv'), index=False)
trainData.to_csv(os.path.join(homePath, 'train_n.csv'), index=False)

In [33]:
# allData.to_csv(os.path.join(homePath, 'train_a.csv'), index=False)